# Tratamento dos dados a serem utilizados para o desafio - 
    ## Machine Learning

In [3]:
import os
import sys
import pandas as pd
notebook_dir = os.getcwd() 
project_root = os.path.dirname(notebook_dir) 
source_path = os.path.join(project_root, 'source')

if source_path not in sys.path:
    sys.path.append(source_path)

from Funções_tratar_dados.tratar_dados import *

df = pd.read_csv(r'C:/Users/Brunin/Documents/Machine_Learning_random_Forest/dados/dados_brutos/Titanic2-Dataset.csv')
display(df.head())

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
coluna_embarled = df['Embarked'].isnull().sum()
print(f'Número de valores nulos na coluna Embarked: {coluna_embarled}')
df['Embarked'] = df['Embarked'].map ({'S' : 0 , 'C' : 1 , 'Q' : 2 })
display(df.head())

Número de valores nulos na coluna Embarked: 2


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,0.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,1.0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,0.0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,0.0



### É preciso trocar a coluna de sex , no caso os gêneros que estão em string , por números  

In [8]:
df['Sex'] = df['Sex'].map({'male' : 0 , 'female' : 1 })
display(df.head())

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,0.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,1.0
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,0.0
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,0.0


In [ ]:
pasta_destino_relativa = '../dados/dados_processados' 
nome_arquivo = 'titanic_tratado_final.csv'
os.makedirs(pasta_destino_relativa, exist_ok=True)
df.to_csv(os.path.join(pasta_destino_relativa, nome_arquivo), index=False)
caminho_absoluto = os.path.abspath(os.path.join(pasta_destino_relativa, nome_arquivo))
print(f"Exportado com sucesso para: {caminho_absoluto}")

Exportado com sucesso para: c:\Users\Brunin\Documents\Machine_Learning_random_Forest\dados\dados_processados\titanic_tratado_final.csv


### Implementação do método random forest ( Floresta aleatória )

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score  , classification_report

features = ['Pclass' , 'Age' , 'SibSp' , 'Parch' ,'Fare' , 'Sex' , 'Embarked']
X = df[features]
Y = df['Survived']
X_train , X_test , Y_train , Y_test = train_test_split (X ,Y , test_size  = 0.3 , random_state = 42 , stratify = Y)
print (f'Shape de x_TREINO {X_train.shape }')
print (f'Shape de x_TESTE {X_test.shape }')
modelo_random_forest = RandomForestClassifier (
    n_estimators = 100 , max_depth = 5 , random_state = 42
)

print ('Iniciando o aperfeiçoamento do modelo...')
modelo_random_forest.fit (X_train , Y_train)
print ('Aperfeiçoamento concluído!')
previsoses = modelo_random_forest.predict(X_test)
acuracia = accuracy_score(Y_test , previsoses)
print (f'-------- Resultados---------')
print (f'Acurácia do modelo : {acuracia *100 :.2f}')
print ('Relatório de Classificação :')
print (classification_report(Y_test , previsoses))

variaveis_importantes = pd.Series (modelo_random_forest.feature_importances_ , index = features).sort_values(ascending = False)
print ('Variáveis mais importantes para o modelo :')
print (variaveis_importantes)

Shape de x_TREINO (623, 7)
Shape de x_TESTE (268, 7)
Iniciando o aperfeiçoamento do modelo...
Aperfeiçoamento concluído!
-------- Resultados---------
Acurácia do modelo : 81.72
Relatório de Classificação :
              precision    recall  f1-score   support

           0       0.81      0.92      0.86       165
           1       0.83      0.66      0.74       103

    accuracy                           0.82       268
   macro avg       0.82      0.79      0.80       268
weighted avg       0.82      0.82      0.81       268

Variáveis mais importantes para o modelo :
Sex         0.417101
Fare        0.190432
Pclass      0.139638
Age         0.135294
SibSp       0.041942
Parch       0.038950
Embarked    0.036644
dtype: float64
